# Summary

Test out new batch query functionality.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import openai
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend, query_kwargs_grid
from htools import *

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [6]:
gpt = GPTBackend()
gpt.switch('repeat')

Switching openai backend to "repeat".


In [11]:
def test_backend(backend, results=None):
    def gen():
        yield from results.kwargs
        
    with gpt(backend):
        kwargs_list = []
        resp_list = []
        gen_kwargs = gen if results else query_kwargs_grid
        for i, kwargs in enumerate(gen_kwargs(), start=1):
            if results:
                res = results.res[i - 1]
            else:
                res = gpt.query(**kwargs)
            print(f'\n\n{i}.')
            if kwargs['stream']:
                cur = []
                for tok, tok_full in res:
                    cur.append((tok, tok_full))
                    print(tok)
                    print('\t' + str(tok_full) + '\n')
                    if tok_full['finish_reason']: print('\n---\n')
            else:
                texts, fulls = res
                print('TEXTS:', texts)
                print('FULLS:', fulls)
                cur = res
            print(spacer())

            kwargs_list.append(kwargs)
            resp_list.append(cur)
    return Results(kwargs=kwargs_list, res=resp_list)

Issues

- no prompt_index in stream=True mode for either repeat or banana (pretty sure not for paid backends either)

In [12]:
repeat_res = test_backend('repeat')

Switching openai backend to "repeat".
np>1: True
nc>1: True
stream: True
{'n': 2, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
{'n': 2, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'How many', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}


1.
YESTERDAY 
	{'index': 0, 'finish_reason': None}

WAS 
	{'index': 0, 'finish_reason': 'dummy'}


---

YESTERDAY 
	{'index': 1, 'finish_reason': None}

WAS 
	{'index': 1, 'finish_reason': 'dummy'}


---

HOW 
	{'index': 2, 'finish_reason': None}

MANY 
	{'index': 2, 'finish_reason': 'dummy'}


---

HOW 
	{'index': 3, 'finish_reason': None}

MANY 
	{'index': 3, 'finish_reason': 'dummy'}


---


-------------------------------------------------------------------------------

np>1: True
nc>1: True
stream: False
{'n': 2, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'pro

In [13]:
# Even though it's free, try to avoid hitting the API too much just in case 
# they have some rate limit.
# banana_res = test_backend('banana')
banana_res = test_backend('banana', results=banana_res)

Switching openai backend to "banana".


1.
 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': None}

a 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': None}

good 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': None}

day 
	{'id': 'ab2f3b0a-0011-495d-848a-62bc558484a1', 'message': 'success', 'created': 1649736961, 'apiVersion': '26 Nov 2021', 'modelOutputs': [{'output': ' a good day', 'input': 'Yesterday was'}], 'index': 0, 'finish_reason': 'dummy'}


---

 
	{'id': '896ed

In [14]:
banana_res.res[6]

[(' ',
  {'id': 'a21dfd6a-c789-4c7d-841c-a70190d16eef',
   'message': 'success',
   'created': 1649736965,
   'apiVersion': '26 Nov 2021',
   'modelOutputs': [{'output': ' a weird day', 'input': 'Yesterday was'}],
   'index': 0,
   'finish_reason': None}),
 ('a ',
  {'id': 'a21dfd6a-c789-4c7d-841c-a70190d16eef',
   'message': 'success',
   'created': 1649736965,
   'apiVersion': '26 Nov 2021',
   'modelOutputs': [{'output': ' a weird day', 'input': 'Yesterday was'}],
   'index': 0,
   'finish_reason': None}),
 ('weird ',
  {'id': 'a21dfd6a-c789-4c7d-841c-a70190d16eef',
   'message': 'success',
   'created': 1649736965,
   'apiVersion': '26 Nov 2021',
   'modelOutputs': [{'output': ' a weird day', 'input': 'Yesterday was'}],
   'index': 0,
   'finish_reason': None}),
 ('day ',
  {'id': 'a21dfd6a-c789-4c7d-841c-a70190d16eef',
   'message': 'success',
   'created': 1649736965,
   'apiVersion': '26 Nov 2021',
   'modelOutputs': [{'output': ' a weird day', 'input': 'Yesterday was'}],
   'in